# 1) Установим окружение kaggle

In [3]:
!pip install -q -U kaggle_environments

# 2) Импортируем функцию запуска соревнования двух ботов

In [4]:
from kaggle_environments import evaluate

# 3) Боты

## 3.1) Тривиальный бот "всегда камень" и "всегда бумага" (для тестов)

In [5]:
%%writefile rock_only_ag.py

#0 - камень 1 - бумага  2 - ножницы
def rock_only_ag(observation, configuration):
    # параметры не используются
    return 0

Overwriting rock_only_ag.py


In [67]:
%%writefile paper_only_ag.py

#0 - камень; 1 - бумага; 2 - ножницы
def paper_only_ag(observation, configuration):
    # параметры не используются
    return 1

Overwriting paper_only_ag.py


## 3.2) Тривиальный бот "бумага или ножницы"

In [13]:
%%writefile paper_or_scissors_ag.py

#0 - камень 1 - бумага 2 - ножницы
def paper_or_scissors_ag(observation, configuration):
    # параметры не используются
    import random as r
    return r.randint(1,2)
    
#for i in range(1,20): print(paper_or_scissors_ag(1,1))

Overwriting paper_or_scissors_ag.py


## 3.3) Тривиальный бот "случайный бот"

In [17]:
%%writefile random_ag.py

#0 - камень 1 - бумага 2 - ножницы
def random_ag(observation, configuration):
    # параметры не используются
    import random as r
    return r.randint(0,2)

#for i in range(1,20): print(random_ag(1,1))  

Overwriting random_ag.py


## 3.3) Тривиальный бот "homo sapiens"

In [48]:
%%writefile homo_sapiens_ag.py

# данный бот реализует стратегию, которой подсознательно следует человек:
# чаще обычного выбирает камень (44%), реже обычного выбирает ножницы (23%), с вероятностью 33% выбирает бумагу

#0 - камень 1 - бумага 2 - ножницы
def homo_sapiens_ag(observation, configuration):
    # параметры не используются
    import random as r
    x = r.randint(1,100)
    if x < 45:
        return 0
    elif x > 77:
        return 2
    else: 
        return 1
## проверим, что вероятности распределены как мы задумывали:

#rock, paper, scissor = 0, 0, 0
#z = 1000
#for i in range(1,z+1):
#    x = homo_sapiens_ag(1,1)
#    if x == 0: 
#        rock = rock + 1
#    elif x == 1: 
#        paper = paper + 1
#    else:
#        scissor = scissor + 1
#print(round(100*rock/z), round(100*paper/z), round(100*scissor/z))

Overwriting homo_sapiens_ag.py


## 3.4) Тривиальный бот "Анти - homo sapiens"

In [47]:
%%writefile anti_homo_sapiens_ag.py

# данный бот реализует стратегию, направленную на победу при игре с homo sapiens
# чаще обычного выбирает ножницы (44%), реже обычного выбирает камень (23%), с вероятностью 33% выбирает бумагу

#0 - камень 1 - бумага 2 - ножницы
def anti_homo_sapiens_ag(observation, configuration):
    # параметры не используются
    import random as r
    x = r.randint(1,100)
    if x < 45:
        return 2
    elif x > 77:
        return 0
    else: 
        return 1
## проверим, что вероятности распределены как мы задумывали:

#rock, paper, scissor = 0, 0, 0
#z = 100000
#for i in range(1,z+1):
#    x = anti_homo_sapiens_ag(1,1)
#    if x == 0: 
#        rock = rock + 1
#    elif x == 1: 
#        paper = paper + 1
#    else:
#        scissor = scissor + 1
#print(round(100*rock/z), round(100*paper/z), round(100*scissor/z))

Writing anti_homo_sapiens_ag.py


# 3.5) Бот "серийный рандом" - не использовать, не готов

In [200]:
%%writefile random_series.py

# этот бот помнит свои предыдущие действия
# бот выбирает два случайных числа
#    x: от 1 до 3
#    y: от 3 до 7
# и выбрасывает (x-1) подряд y раз, потом снова выбрасывает два числа...

#0 - камень 1 - бумага 2 - ножницы
def random_series(observation, configuration):
    # параметры не используются
    import random as r
    x = r.randint(1, 3)
    y = r.randint(3, 7)
    global serv_int
    global serv_int1
    # если это первый цикл или если предыдущий цикл закончился, то сохраняем параметры ближайших запусков
    if serv_int1 == 1 or serv_int1 == 0:
        serv_int = x # что возвращаем + 1
        serv_int1 = y # сколько шагов к ряду возвращаем x-1
        #print(serv_int-1, serv_int1 )
        return serv_int-1
    # если не первый цикл, значит serv_int уже определен, вывдем его, уменьшив число оставшихся шагов на 1
    if serv_int1 != 1 and serv_int1 != 0:
        serv_int1 = serv_int1 - 1
        #print(serv_int-1, serv_int1 )
        return serv_int-1
    
#random_series(1,1)


Overwriting random_series.py


In [199]:
import pandas as pd 

# схватку ботов оформим функцией:
def eval_agents(rew_threshld=1, step_num=2, ag_1='', ag_2=''):

    x = evaluate(
    configuration = {'tieRewardThreshold': rew_threshld, 'episodeSteps':step_num + 1 },
    environment = "rps", #выбираем окружение КНБ
    agents = [ag_1 + '.py', ag_2 + '.py']
    ) 
    return x

# определим параметры битвы ботов
steps = 100 # сколько шагов в одной серии
rewardthreshold = 1 # сколько раз нужно победить, чтобы получить 1 очко

# создадим массив всех наших ботов
ag_list = ['rock_only_ag','paper_only_ag','paper_or_scissors_ag', 'random_ag','homo_sapiens_ag','anti_homo_sapiens_ag']

# запускаем схватку всех ботов со всеми и формируем датафрейм с результатами

# полный списко результатов
res_dict = {}
for i in ag_list: # это колонки таблицы, первый агент в колонках.
    res_dict[i] = list()
    for j in ag_list: # это строки таблицы, второй агент в индексе строк
        res_dict[i] = res_dict[i] + eval_agents(rewardthreshold, steps, i, j )
        
# сокращенный списко результатов        
res_short_dict = {}
for x in res_dict:
    ag_res_list = []
    for y in res_dict[x]:
        ag_res_list = ag_res_list +  [y[0]]
    res_short_dict[x] = ag_res_list


## Выводим результат в двух вариантах:

df_res = pd.DataFrame(res_dict, index = ag_list)
print('-----------------------------------')
print('------------Full results-----------')
print('Fight count: ' + str(steps))
print('Number of bots:' + str(len(ag_list)))
print('First agent - column name, second agent - row name:')
display(df_res)

df_short_res = pd.DataFrame(res_short_dict, index = ag_list)
print('-----------------------------------')
print('------------Short results-----------')
print('Fight count: ' + str(steps))
print('Number of bots:' + str(len(ag_list)))
print('Values in table show number figts first agent (column name) win (>0) or lose (<0) with second agent (row name).\
Zero means they are equally strong.')
display(df_short_res)



-----------------------------------
------------Full results-----------
Fight count: 100
Number of bots:6
First agent - column name, second agent - row name:


,rock_only_ag,paper_only_ag,paper_or_scissors_ag,random_ag,homo_sapiens_ag,anti_homo_sapiens_ag
rock_only_ag,"[0, 0]","[100.0, -100.0]","[0, 0]","[8.0, -8.0]","[3.0, -3.0]","[-13.0, 13.0]"
paper_only_ag,"[-100.0, 100.0]","[0, 0]","[51.0, -51.0]","[1.0, -1.0]","[-22.0, 22.0]","[15.0, -15.0]"
paper_or_scissors_ag,"[4.0, -4.0]","[-49.0, 49.0]","[5.0, -5.0]","[-3.0, 3.0]","[-19.0, 19.0]","[16.0, -16.0]"
random_ag,"[8.0, -8.0]","[-1.0, 1.0]","[0, 0]","[-10.0, 10.0]","[7.0, -7.0]","[-1.0, 1.0]"
homo_sapiens_ag,"[-3.0, 3.0]","[17.0, -17.0]","[5.0, -5.0]","[-3.0, 3.0]","[-2.0, 2.0]","[2.0, -2.0]"
anti_homo_sapiens_ag,"[21.0, -21.0]","[-8.0, 8.0]","[-7.0, 7.0]","[1.0, -1.0]","[6.0, -6.0]","[-4.0, 4.0]"


-----------------------------------
------------Short results-----------
Fight count: 100
Number of bots:6
Values in table show number figts first agent (column name) win (>0) or lose (<0) with second agent (row name).Zero means they are equally strong.


,rock_only_ag,paper_only_ag,paper_or_scissors_ag,random_ag,homo_sapiens_ag,anti_homo_sapiens_ag
rock_only_ag,0.0,100.0,0.0,8.0,3.0,-13.0
paper_only_ag,-100.0,0.0,51.0,1.0,-22.0,15.0
paper_or_scissors_ag,4.0,-49.0,5.0,-3.0,-19.0,16.0
random_ag,8.0,-1.0,0.0,-10.0,7.0,-1.0
homo_sapiens_ag,-3.0,17.0,5.0,-3.0,-2.0,2.0
anti_homo_sapiens_ag,21.0,-8.0,-7.0,1.0,6.0,-4.0


In [ ]:
    
    #in case we have information about opponent last move
   # if observation.step > 0:
    #    return observation.lastOpponentAction
    #initial step
   # else:
    #    return random.randrange(0, configuration.signs)
    

In [71]:
import numpy as np
import pandas as pd
import random as rn

#import matplotlib.pyplot as plt
#import seaborn as sns

from kaggle_environments import make, evaluate

for i in range(2,100):
    conf = {"episodeSteps": i}
    print(
            evaluate(
                "rps", #environment to use - no need to change
                ["rock_only_ag.py", "paper_only_ag.py"], #первый агент всегда камень, второй всегда бумага
                configuration=conf
            )
    )


[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[0, 0]]
[[-20.0, 20.0]]
[[-21.0, 21.0]]
[[-22.0, 22.0]]
[[-23.0, 23.0]]
[[-24.0, 24.0]]
[[-25.0, 25.0]]
[[-26.0, 26.0]]
[[-27.0, 27.0]]
[[-28.0, 28.0]]
[[-29.0, 29.0]]
[[-30.0, 30.0]]
[[-31.0, 31.0]]
[[-32.0, 32.0]]
[[-33.0, 33.0]]
[[-34.0, 34.0]]
[[-35.0, 35.0]]
[[-36.0, 36.0]]
[[-37.0, 37.0]]
[[-38.0, 38.0]]
[[-39.0, 39.0]]
[[-40.0, 40.0]]
[[-41.0, 41.0]]
[[-42.0, 42.0]]
[[-43.0, 43.0]]
[[-44.0, 44.0]]
[[-45.0, 45.0]]
[[-46.0, 46.0]]
[[-47.0, 47.0]]
[[-48.0, 48.0]]
[[-49.0, 49.0]]
[[-50.0, 50.0]]
[[-51.0, 51.0]]
[[-52.0, 52.0]]
[[-53.0, 53.0]]
[[-54.0, 54.0]]
[[-55.0, 55.0]]
[[-56.0, 56.0]]
[[-57.0, 57.0]]
[[-58.0, 58.0]]
[[-59.0, 59.0]]
[[-60.0, 60.0]]
[[-61.0, 61.0]]
[[-62.0, 62.0]]
[[-63.0, 63.0]]
[[-64.0, 64.0]]
[[-65.0, 65.0]]
[[-66.0, 66.0]]
[[-67.0, 67.0]]
[[-68.0, 68.0]]
[[-69.0, 69.0]]
[[-70.0, 70.0]]
[[-71.0, 71.0